In [ ]:
import os
import random
from datetime import datetime

now = datetime.now()

# Folder name to save registration transforms
prefix = "lassoTol5Fold"
resultsFolder = "results/" + prefix + now.strftime("%Y_%m_%d_%H_%M_%S")
print("Results folder name:", resultsFolder)

path = "./Dataset/data"
labelPath = "Dataset/labels"

data = os.listdir(path)
datasetLen = len(data)

# random shuffle data
random.shuffle(data)


"""Write transforms."""
from pathlib import Path


try:
    Path(f"./{resultsFolder}").mkdir(parents=True, exist_ok=True)
    print("Saving data names.")
    file = open(f"./{resultsFolder}/data.txt", "w") 
    for i, name in enumerate(data):
        # print(f"\tSaving {name}. {i+1}/{len(trainData)}", end='\x1b[1K\r')
        # print(f"\tSaving {name}. {i+1}/{len(trainData)}")
        file.write(name+"\n")
    # print()
    file.close()
except:
    print("Can't save names.")


# Fork processes for each test image
# Doesn't work on jupyter notebook.
# Run it on python.
testImageIdx = 0
for i in range(1, len(data)):
# for i in range(1, 0): # DEBUG 
    n = os.fork() 
    if n == 0: 
        # child
        testImageIdx = i
        break
        
# fill trainData according to the fold
    
fold = testImageIdx // (len(data)//5)
fold = 4 if fold > 4 else fold

trainData = []
for i in range(len(data)):
    index = i // (len(data)//5)
    index = 4 if index > 4 else index
    if index != fold:
        trainData.append(data[i])

resultsFolder += f"/{testImageIdx:02d}"
try:
    Path(f"./{resultsFolder}").mkdir(parents=True, exist_ok=True)
except:
    print("Can't make the results directory.")


In [ ]:
UseCurvatureFlow = True
timeStep = 0.04
numberOfIterations = 10
totalLabels = 2

# Tests
lassoTols = [0.1, 0.01, 0.001, 0.0001]
atlasesNums = [1]
Ps = [[5, 5, 5]]
Ns = [[5, 5, 5]]

# Ns = [[7, 7, 7], [5, 5, 5], [3, 3, 3]]
# Ps = [[3, 3, 3], [5, 5, 5], [7, 7, 7]]
# lassoTols = [0.01]

inputData = []
for atlas in atlasesNums:
    for P in Ps:
        for N in Ns:
            for lassoTol in lassoTols:
                inputData.append((atlas, P, N, lassoTol))
                

In [ ]:
import SimpleITK as sitk
import numpy as np
from utils import *

# Set the number of threads
sitk.ImageRegistrationMethod().SetGlobalDefaultNumberOfThreads(1)

transforms = []
mseList = []

# find biggest P and N
P = np.argmax(Ps, axis=0)[0]
N = np.argmax(Ns, axis=0)[0]

# Read fixed image
pathName = os.path.join(path, data[testImageIdx])
if UseCurvatureFlow:
    fixedImage = sitk.CurvatureFlow(sitk.ReadImage(pathName), 
                                    timeStep = timeStep, 
                                    numberOfIterations = numberOfIterations)
else:
    fixedImage = sitk.ReadImage(pathName)


# for i, f in enumerate(trainData[:1]): # DEBUG
for i, f in enumerate(trainData):
    # print(f'Registration of {f}. {i+1}/{len(trainData)}', end='\x1b[1K\r')
    print(f'Registration of {f}. {i+1}/{len(trainData)}')

    pathName = os.path.join(path, f)

    # Histogram matching
    if UseCurvatureFlow:
        image = sitk.HistogramMatching(sitk.CurvatureFlow(sitk.ReadImage(pathName),
                                                          timeStep = timeStep, 
                                                          numberOfIterations = numberOfIterations),
                                       fixedImage)
    else:
        image = sitk.HistogramMatching(sitk.ReadImage(pathName), fixedImage)

    label = sitk.ReadImage(os.path.join(labelPath, f))

    # t = registration(fixedImage, image, label)
    # t = registration2(fixedImage, image, label)
    # t = registrationElastix(fixedImage, image, resultsFolder)
    # t = registrationElastix2(fixedImage, image, resultsFolder)
    t = registrationElastixMask(fixedImage, image, label, resultsFolder)


    # add transform to the list
    transforms.append(t)

    # add mse to the list
    # mseList.append(mse3D(fixedImage, resampleImage(image, fixedImage, t)))
    mseList.append(mse3DLabels(fixedImage, 
                               resampleImage(image, fixedImage, t), 
                               resampleLabels(label, fixedImage, t)))

# sort mseList
mseListSorted = sorted(enumerate(mseList), key=lambda x: x[1])

###############################################################################
# For every test
###############################################################################
for testIndex, (atlasesNum, P, N, lassoTol) in enumerate(inputData):
# for testIndex, (atlasesNum, P, N, lassoTol) in enumerate(inputData[0:1]): # DEBUG
    # pick the desired number of atlases
    minx = miny = minz = 10000
    maxx = maxy = maxz = 0
    images = []
    labels = []

    # Read fixed image
    pathName = os.path.join(path, data[testImageIdx])
    if UseCurvatureFlow:
        fixedImage = sitk.CurvatureFlow(sitk.ReadImage(pathName), 
                                        timeStep = timeStep, 
                                        numberOfIterations = numberOfIterations)
    else:
        fixedImage = sitk.ReadImage(pathName)

    ###############################################################################
    # Read atlases and labels and tranform them
    ###############################################################################
    for i in range(atlasesNum):
    # for i in range(1): # DEBUG
        idx = mseListSorted[i][0]
        t = transforms[idx]
        f = trainData[idx]
        pathName = os.path.join(path, f)
        # print(f'Reading {f}. {i+1}/{atlasesNum}', end='\x1b[1K\r')
        print(f'Reading {f}. {i+1}/{atlasesNum}')

        # Histogram matching
        if UseCurvatureFlow:
            images.append(sitk.GetArrayFromImage(resampleImage(
                        sitk.HistogramMatching(
                            sitk.CurvatureFlow(sitk.ReadImage(pathName), 
                                               timeStep = timeStep, 
                                               numberOfIterations = numberOfIterations),
                                            fixedImage), 
                          fixedImage, t)))
        else:
            images.append(sitk.GetArrayFromImage(resampleImage(
                        sitk.HistogramMatching(
                            sitk.ReadImage(pathName),
                            fixedImage),  
                        fixedImage, t)))

        images[i][images[i] < 0] = 0
        images[i] = np.asarray(images[i], dtype=np.uint16)

        label = sitk.ReadImage(os.path.join(labelPath, f))
        label = sitk.GetArrayFromImage(resampleLabels(label, fixedImage, t))
        labels.append(label)

        idxs = np.nonzero(label != 0)
        minx = min([minx, min(idxs[0])])
        maxx = max([maxx, max(idxs[0])])
        miny = min([miny, min(idxs[1])])
        maxy = max([maxy, max(idxs[1])])
        minz = min([minz, min(idxs[2])])
        maxz = max([maxz, max(idxs[2])])


    ###############################################################################
    # Calculate the desired shape
    ###############################################################################
    shape, copyShape, offset, length = calculateCropShape(images, P, N, minx, maxx, miny, maxy, minz, maxz)

    ###############################################################################
    # crop images and labels
    ###############################################################################
    for i in range(len(images)):
        images[i] = cropImage(images[i], shape, offset, length, copyShape, "uint16")
        labels[i] = cropImage(labels[i], shape, offset, length, copyShape, "uint8")

    fixedImage = sitk.GetArrayFromImage(fixedImage)
    fixedImage = cropImage(fixedImage, shape, offset, length, copyShape, "uint16")


    ###############################################################################
    # Make to the proper type
    ###############################################################################
    # print("Fixing data types.", end='\x1b[1K\r')
    print("Fixing data types.")
    images = np.array(images, order='C')
    labels = np.array(labels, order='C')
    P = np.array(P, dtype=np.int32)
    N = np.array(N, dtype=np.int32)

    ###############################################################################
    # Perform segmentation
    ###############################################################################
    from cGen import cGen
    print("Running segmentation.")
    segmentationSPBM, segmentationSRC = cGen.applySPBMandSRCSpams(fixedImage, images, 
                                  labels, totalLabels, P, N, lassoTol=lassoTol, 
                                  lassoMaxIter=1e4, verboseX=True,  verboseY=False, 
                                  xmin=-1, xmax=-1, ymin=-1, ymax=-1, zmin=-1, zmax=-1,
                                  # xmin=10, xmax=11, ymin=10, ymax=11, zmin=10, zmax=11,
                                  numThreads=1, lassoL=-1)

    ###############################################################################
    # Save segmentation results
    ###############################################################################

    # print("Saving results.", end='\x1b[1K\r')
    print("Saving results.")
    try:
        saveSegmentation(segmentationSPBM,
                         os.path.join(path, data[testImageIdx]), 
                         os.path.join(resultsFolder, data[testImageIdx]) + f"SPBM_{testIndex}.mhd", 
                         copyShape, 
                         offset,
                         length,
                         verbose=False)
    except:
        print("Can't save SPBM result.")

    try:
        saveSegmentation(segmentationSRC,
                         os.path.join(path, data[testImageIdx]), 
                         os.path.join(resultsFolder, data[testImageIdx]) + f"SRC_{testIndex}.mhd", 
                         copyShape, 
                         offset,
                         length,
                         verbose=False)
    except:
        print("Can't save SRC result.")


    ###############################################################################
    # Save dice index
    ###############################################################################

    originalLabel = sitk.GetArrayFromImage(sitk.ReadImage(os.path.join(labelPath, data[testImageIdx])))

    translatedSegmentationSPBM = translateToOriginal(segmentationSPBM, 
                                                     os.path.join(path, data[testImageIdx]), 
                                                     copyShape, 
                                                     offset, 
                                                     length)

    diceSPBM = Dice(originalLabel, translatedSegmentationSPBM, totalLabels)

    translatedSegmentationSRC = translateToOriginal(segmentationSRC, 
                                                    os.path.join(path, data[testImageIdx]), 
                                                    copyShape, 
                                                    offset, 
                                                    length)

    diceSRC = Dice(originalLabel, translatedSegmentationSRC, totalLabels)

    print(f"{testIndex}: Atlases: {atlasesNum}, P: {P}, N: {N}, lassoTol: {lassoTol}")
    print("SPBM:", diceSPBM)
    print("SRC:", diceSRC)

    try:
        diceFile = open(f"./{resultsFolder}/dice.txt", "a") 
        diceFile.write(f"{testIndex}: Atlases: {atlasesNum}, P: {P}, N: {N}, lassoTol: {lassoTol}"+"\n")
        diceFile.write("SPBM: " + str(diceSPBM) + "\n")
        diceFile.write("SRC: " + str(diceSRC) + "\n")
        diceFile.close()
    except:
        print("Can't append dice results.")

In [ ]:
####################################################
# END OF NORMAL CODE 
####################################################
import sys
sys.exit()

In [ ]:

pathName = os.path.join(path, "9036287.nii.gz")
image = sitk.ReadImage(pathName)
print(image.GetPixelID())
test = sitk.GetArrayFromImage(image)
test = test.astype(np.uint16)

newImage = sitk.GetImageFromArray(test)
newImage.CopyInformation(image)
print(newImage.GetPixelID())
pathName = os.path.join(path,  "9036287.nii.gz")
# sitk.WriteImage(newImage, pathName, True)

In [ ]:
import numpy as np

In [ ]:
sitk.GetArrayFromImage(image)

In [ ]:

(((sitk.GetArrayFromImage(image) - sitk.GetArrayFromImage(fixedImage))[sitk.GetArrayFromImage(label) > 0])**2).mean()

In [ ]:
((sitk.GetArrayFromImage(image) - sitk.GetArrayFromImage(fixedImage))**2).mean()


In [ ]:
mask = sitk.GetArrayFromImage(label)
mask[mask > 1] = 1
print(((mask * (sitk.GetArrayFromImage(image) - sitk.GetArrayFromImage(fixedImage)))**2).sum())
((mask * sitk.GetArrayFromImage(image) - mask * sitk.GetArrayFromImage(fixedImage))**2).sum()

In [ ]:

for testIndex, (atlasesNum, P, N, lassoTol) in enumerate(inputData):
    print(testIndex, atlasesNum, P, N, lassoTol)

In [ ]:
from PIL import Image
import numpy as np

def showImg_(img, z=60):
    a = sitk.GetArrayFromImage(img[z,:,:])
    disImg = Image.fromarray(np.interp(a, (a.min(), a.max()), (0, 255)).astype('uint8'))
    disImg.show()
    
def showImg2_(img, z=60):
    disImg = Image.fromarray(sitk.GetArrayFromImage(img[z,:,:]).astype('uint8'))
    disImg.show()
    
def showImg3_(img, z=60):
    a = sitk.GetArrayFromImage(img[z,:,:])
    disImg = image.fromarray(np.interp(a, (0, 5000), (0, 255)).astype('uint8'))
    disImg.show()
    
def showImg4_(img, z=60):
    img2 = Image.fromarray(sitk.GetArrayFromImage(img[z,:,:])*60, 'L')
    img2.show()

In [ ]:
labels[0].shape

In [ ]:
showImg2_(sitk.GetImageFromArray(fixedImage), 20)
showImg2_(sitk.GetImageFromArray(images[0]), 20)
showImg_(sitk.GetImageFromArray(labels[0]), 20)
# showImg_(sitk.GetImageFromArray(segmentationSPBM), 20)

In [ ]:
showImg2_(sitk.GetImageFromArray(fixedImage), 60)
showImg2_(sitk.GetImageFromArray(images[0]), 60)
showImg_(sitk.GetImageFromArray(segmentationSPBM), 60)
# showImg2_(images[1], 60)

In [ ]:
segmentationSPBM[segmentationSPBM > 0]

In [ ]:
l = sitk.GetArrayFromImage(label)
l[l == 4] = 0
mask = sitk.GetImageFromArray(l)

showImg4_(mask, 100)


In [ ]:
sitk.ImageRegistrationMethod().SetGlobalDefaultNumberOfThreads()

In [ ]:
help(sitk.ElastixImageFilter())
# sitk.ElastixImageFilter().GetOutputDirectory()

In [ ]:
help(elastixImageFilter)

In [ ]:
import SimpleITK as sitk
import os
from utils import *

path = "./Dataset/data"
labelPath = "Dataset/labels"

data = os.listdir(path)
datasetLen = len(data)

# spilt dataset
trainData = data[:int(datasetLen * 0.7)]
testData = data[int(datasetLen * 0.7):]

"""Write transforms."""
from pathlib import Path

timeStep = 0.04
numberOfIterations = 10

pathName = os.path.join(path, testData[0])
# fixedImage = sitk.ReadImage(pathName)

# fixedImage = sitk.CurvatureFlow(sitk.ReadImage(pathName), timeStep = 0.04, numberOfIterations = 10)
fixedImage = sitk.CurvatureFlow(sitk.ReadImage(pathName), timeStep = timeStep,
                                numberOfIterations = numberOfIterations)


for i, f in enumerate(trainData[8:9]): # for test
# for i, f in enumerate(trainData):
    # print(f'Registration of {f}. {i+1}/{len(trainData)}', end='\x1b[1K\r')
    print(f'Registration of {f}. {i+1}/{len(trainData)}')
    
    pathName = os.path.join(path, f)
                  
    # image = sitk.ReadImage(pathName)
    
    # image = sitk.HistogramMatching(sitk.ReadImage(pathName), fixedImage)
    image = sitk.HistogramMatching(sitk.CurvatureFlow(sitk.ReadImage(pathName), 
                                                      # timeStep = 0.04, 
                                                      # numberOfIterations = 10),
                                                      timeStep = timeStep, 
                                                      numberOfIterations = numberOfIterations),
                                   fixedImage)
    
    label = sitk.ReadImage(os.path.join(labelPath, f))
    
    # t = registrationElastixMask(fixedImage, image, label)
    # t2 = registration(fixedImage, image, label)

    resample = sitk.ResampleImageFilter()
    resample.SetReferenceImage(fixedImage)

    resample.SetInterpolator(sitk.sitkLinear)
    # resample.SetTransform(t)
    resample.SetSize()

    newImage = resample.Execute(image)


In [ ]:

resample = sitk.ResampleImageFilter()
resample.SetReferenceImage(fixedImage)

resample.SetInterpolator(sitk.sitkLinear)
# resample.SetTransform(t)
resample.SetSize([i // 2 for i in list(image.GetSize())])
resample.SetOutputSpacing([i * 2 for i in list(image.GetSpacing())])

newImage = resample.Execute(image)

In [ ]:
resample = sitk.ResampleImageFilter()
resample.SetReferenceImage(newImage)

resample.SetInterpolator(sitk.sitkLinear)
# resample.SetTransform(t)
resample.SetSize([i * 2 for i in list(newImage.GetSize())])
resample.SetOutputSpacing([i / 2 for i in list(newImage.GetSpacing())])

image2 = resample.Execute(image)

In [ ]:
shrinkFilter = sitk.ShrinkImageFilter()
shrinkFilter.SetShrinkFactor(1)
newImage2 = shrinkFilter.Execute(image)


expandFilter = sitk.ExpandImageFilter()
expandFilter.SetExpandFactor(1)
expandFilter.SetInterpolator(sitk.sitkLinear)
image2 = expandFilter.Execute(newImage2)


In [ ]:
print(newImage2.GetSize())

showImg_(newImage2, 30)

In [ ]:
size = [i // 2 for i in list(image.GetSize())]
'''
for i, val in enumerate(image.GetSize()):
    print(i, val)
    size[i] = val // 2
'''   
print(size)

In [ ]:
showImg_(image)
showImg_(image2)

In [ ]:
fixedImage.GetPixelID()

In [ ]:
import SimpleITK as sitk
help(image)

In [ ]:
help(sitk.GetDefaultParameterMap("affine"))
elastixImageFilter.PrintParameterMap()

In [ ]:
import SimpleITK as sitk
import os
from utils import *

path = "./Dataset/data"
labelPath = "Dataset/labels"

data = os.listdir(path)
datasetLen = len(data)

# spilt dataset
trainData = data[:int(datasetLen * 0.7)]
testData = data[int(datasetLen * 0.7):]

"""Write transforms."""
from pathlib import Path

timeStep = 0.04
numberOfIterations = 10

pathName = os.path.join(path, testData[0])
# fixedImage = sitk.ReadImage(pathName)

# fixedImage = sitk.CurvatureFlow(sitk.ReadImage(pathName), timeStep = 0.04, numberOfIterations = 10)
fixedImage = sitk.CurvatureFlow(sitk.ReadImage(pathName), timeStep = timeStep,
                                numberOfIterations = numberOfIterations)


for i, f in enumerate(trainData[8:9]): # for test
# for i, f in enumerate(trainData):
    # print(f'Registration of {f}. {i+1}/{len(trainData)}', end='\x1b[1K\r')
    print(f'Registration of {f}. {i+1}/{len(trainData)}')
    
    pathName = os.path.join(path, f)
                  
    # image = sitk.ReadImage(pathName)
    
    # image = sitk.HistogramMatching(sitk.ReadImage(pathName), fixedImage)
    image = sitk.HistogramMatching(sitk.CurvatureFlow(sitk.ReadImage(pathName), 
                                                      # timeStep = 0.04, 
                                                      # numberOfIterations = 10),
                                                      timeStep = timeStep, 
                                                      numberOfIterations = numberOfIterations),
                                   fixedImage)
    
    label = sitk.ReadImage(os.path.join(labelPath, f))
    
    t = registrationElastixMask(fixedImage, image, label)
    t2 = registration(fixedImage, image, label)

    showImg_(resampleImage(image, fixedImage, t))
    showImg_(resampleImage(image, fixedImage, t2))
    showImg_(fixedImage)

'''
# showImg_(fixedImage)
# showImg_(image)

# label = sitk.ReadImage(os.path.join(labelPath, f))
# Create the mask
l = sitk.GetArrayFromImage(label)
l[l>1] = 1
mask = sitk.GetImageFromArray(l)
mask.CopyInformation(label)
    
elastixImageFilter = sitk.ElastixImageFilter()
elastixImageFilter.SetFixedImage(fixedImage)
elastixImageFilter.SetMovingImage(image)
elastixImageFilter.AddMovingMask(mask)
elastixImageFilter.SetParameterMap(sitk.GetDefaultParameterMap("affine"))
elastixImageFilter.SetParameter("Metric", "AdvancedMeanSquares")

# elastixImageFilter.SetParameter("MaximumNumberOfIterations", "8192")
elastixImageFilter.SetParameter("MaximumNumberOfIterations", "2048")

elastixImageFilter.SetParameter("MaximumNumberOfSamplingAttempts", "32")
elastixImageFilter.SetParameter("NumberOfSamplesForExactGradient", "8192")
elastixImageFilter.SetParameter("NumberOfSpatialSamples", "8192")
elastixImageFilter.SetParameter("SamplingPercentage", "0.8")

elastixImageFilter.SetParameter("NumberOfJacobianMeasurements", "10000")
# elastixImageFilter.SetParameter("SelfHessianSmoothingSigma", "0")
elastixImageFilter.SetParameter("UseNormalization", "true")

elastixImageFilter.SetParameter("AutomaticTransformInitialization", "true")
elastixImageFilter.SetParameter("ErodeMovingMask", "false")
elastixImageFilter.SetParameter("CheckNumberOfSamples", "false")
elastixImageFilter.SetParameter("WriteResultImage", "false")
elastixImageFilter.LogToFileOff()

# elastixImageFilter.SetOutputDirectory("AHE")
# elastixImageFilter.SetParameter("RandomSparseMask", "true")

"""
elastixImageFilter.SetParameter("FixedImagePyramid", "FixedRecursiveImagePyramid")
elastixImageFilter.SetParameter("MovingImagePyramid", "MovingRecursiveImagePyramid")
elastixImageFilter.SetParameter("ImagePyramidSchedule", "8 8 8  4 4 4  2 2 2  1 1 1")
(FixedImagePyramid "FixedRecursiveImagePyramid")
(MovingImagePyramid "MovingRecursiveImagePyramid")
(ImagePyramidSchedule 8 8 8  4 4 4  2 2 2  1 1 1)
"""

# elastixImageFilter.SetParameter("NumberOfBandStructureSamples", "100")
# elastixImageFilter.SetParameter("ImagePyramidSchedule", "[[1,1,1],[1,1,1],[1,1,1],[1,1,1]]")
# elastixImageFilter.SetParameter("ImagePyramidSchedule", "1")
# elastixImageFilter.SetParameter("AutomaticTransformInitializationMethod", "CenterOfGravity")
# elastixImageFilter.SetParameter("ImageSampler", "RandomSparseMask")
# elastixImageFilter.SetParameter("AutomaticScalesEstimation", "false")

elastixImageFilter.Execute()


showImg_(elastixImageFilter.GetResultImage())
showImg_(fixedImage)

'''


In [ ]:
# resultImage2 = elastixImageFilter.GetResultImage()
transformParameterMap = elastixImageFilter.GetTransformParameterMap()

In [ ]:
showImg_(elastixImageFilter.GetResultImage())

In [ ]:
print(transformParameterMap[0].asdict())

In [ ]:
af = sitk.AffineTransform(3)
# af.SetMatrix([float(i) for i in transformParameterMap[0].asdict()['TransformParameters']])
# af.SetTranslation([float(i) for i in transformParameterMap[0].asdict()['TransformParameters']])
test = [float(i) for i in transformParameterMap[0].asdict()['TransformParameters']]
# test2 = [[i for i in test[0:3]], [i for i in test[4:7]], [i for i in test[8:11]]]
af.SetTranslation(test[9:])
af.SetMatrix(test[0:9])
test = [float(i) for i in transformParameterMap[0].asdict()['CenterOfRotationPoint']]
af.SetCenter(test)

resample = sitk.ResampleImageFilter()
resample.SetReferenceImage(fixedImage)
resample.SetInterpolator(sitk.sitkNearestNeighbor)
resample.SetTransform(af)

testL = resample.Execute(label)

# from utils import *
# testL = resampleLabels(label, fixedImage, af)

resample = sitk.ResampleImageFilter()
resample.SetReferenceImage(fixedImage)
resample.SetInterpolator(sitk.sitkLinear)
resample.SetTransform(af)
testI = resample.Execute(image)

print(resample.GetTransform())

In [ ]:
# help(sitk.ResampleImageFilter())
print(af.GetMatrix())
print(af.GetCenter())
print(af.GetTranslation())
help(af)

In [ ]:
showImg_(elastixImageFilter.GetResultImage())
# showImg_(image)
showImg_(fixedImage)
# showImg_(testL)
# showImg_(testI)
# showImg_(label)
# TODO: maybe deformation fiead????

In [ ]:
sitk.GetArrayFromImage(testL)

In [ ]:
transformixImageFilter = sitk.TransformixImageFilter()
transformixImageFilter.SetTransformParameter("UseBinaryFormatForTransformationParameters", "true")
transformixImageFilter.SetTransformParameterMap(elastixImageFilter.GetTransformParameterMap())
transformixImageFilter.SetMovingImage(label)
transformixImageFilter.AddTransformParameter("Interpolator", "NearestNeighborInterpolator")
transformixImageFilter.SetTransformParameter("ResultImagePixelType", "double")
transformixImageFilter.SetTransformParameter("MovingInternalImagePixelType", "double")
transformixImageFilter.SetTransformParameter("FixedInternalImagePixelType", "double")
# transformixImageFilter.SetTransformParameter("FixedInternalImagePixelType", "sitk.sitkInt8")
# transformixImageFilter.AddTransformParameter("ResampleInterpolator", "NearestNeighborInterpolator")
lel = transformixImageFilter.Execute()
showImg_(lel)

In [ ]:
# Transform label map using the deformation field from above
resultLabel = sitk.Cast(sitk.Transformix(label, elastixImageFilter.GetTransformParameterMap()), sitk.sitkInt8)
showImg_(resultLabel)
np.unique(sitk.GetArrayFromImage(resultLabel))

In [ ]:
sitk.sitkInt8

In [ ]:
np.unique(sitk.GetArrayFromImage(lel))

In [ ]:
transformixImageFilter.PrintParameterMap()

In [ ]:
print(transformixImageFilter.GetComputeSpatialJacobian())
print(transformixImageFilter.GetComputeDeterminantOfSpatialJacobian())
print(transformixImageFilter.GetComputeDeformationField())

In [ ]:
help(sitk.Transformix)

In [ ]:
help(sitk.TransformixImageFilter())

In [ ]:
help(sitk.ElastixImageFilter())

In [ ]:
test = sitk.ElastixImageFilter()
test.AddParameterMap(transformParameterMap)

In [ ]:
sitk.PrintParameterMap(elastixImageFilter.GetParameterMap())

In [ ]:
help(sitk.ElastixImageFilter)

In [ ]:
print(transformParameterMap)

In [ ]:
showImg2_(fixedImage, 60)
showImg2_(images[0], 60)
showImg2_(resultImage, 60)

In [ ]:
showImg_(fixedImage, 60)
showImg_(images[1], 60)
showImg_(resultImage, 60)

In [ ]:
help(sitk.GetImageFromArray)

In [ ]:

labels = np.array(labels, order='C', dtype=np.uint8)

segmentationSPBM, segmentationSRC = cGen.applySPBMandSRCSpams(fixedImage, images, 
                              labels, 3, P, N, lassoTol=0.00001, 
                              lassoMaxIter=1e4, verboseX=True,  verboseY=False, 
                              xmin=-1, xmax=-1, ymin=-1, ymax=-1, zmin=-1, zmax=-1)

In [ ]:
segmentationSPBM, segmentationSRC = cGen.applySPBMandSRCSpams(fixedImage,
                                                         images, 
                              np.ones(labels.shape, dtype=np.uint8, order='C')+50, 3, P, N, lassoTol=0.00001, 
                              lassoMaxIter=1e4, verboseX=True,  verboseY=False, 
                              xmin=-1, xmax=-1, ymin=-1, ymax=-1, zmin=-1, zmax=-1)

In [ ]:
print(fixedImage.shape)
print(fixedImage.dtype)
print(type(fixedImage))
print(images.shape)
print(images.dtype)
print(type(images))
print(labels.shape)
print(labels.dtype)
print(type(labels))

In [1]:
from cGen import cGen
import numpy as np

P = [7, 7, 7]
N = [7, 7, 7]
P = [5, 5, 5]
N = [5, 5, 5]
P = np.array(P, dtype=np.int32)
N = np.array(N, dtype=np.int32)

ii = np.array(np.random.randint(0, high=500, size=(390, 276, 129)), dtype=np.uint16, order='C')
i = np.array(np.random.randint(0, high=500, size=(5, 390, 276, 129)), dtype=np.uint16, order='C')
l = np.array(np.random.randint(0, high=3, size=(5, 390, 276, 129)), dtype=np.uint8, order='C')
# segmentationSPBM, segmentationSRC = cGen.applySPBMandSRCSpams(ii,
'''
segmentationSPBM, segmentationSRC = cGen.applySPBMandSRCSpams(ii,
                                                              i, 
                                                              l, 3, P, N, lassoTol=0.1, 
                              lassoMaxIter=1e0, verboseX=True,  verboseY=False, 
                              xmin=-1, xmax=-1, ymin=-1, ymax=-1, zmin=60, zmax=61,
                              # xmin=-1, xmax=-1, ymin=60, ymax=61, zmin=60, zmax=61,
                                  numThreads=1, lassoL=1)
'''
segmentation = cGen.applySPEP(ii, i,  l, 3, P, N, verboseX=True,  verboseY=True, 
                              xmin=-1, xmax=-1, ymin=-1, ymax=-1, zmin=-1, zmax=-1,)

	X: 4 	Y: 4 	Time: 0.06315970420837402
	X: 4 	Y: 5 	Time: 0.06229352951049805
	X: 4 	Y: 6 	Time: 0.06695413589477539
	X: 4 	Y: 7 	Time: 0.06353354454040527
	X: 4 	Y: 8 	Time: 0.06591343879699707
	X: 4 	Y: 9 	Time: 0.06320071220397949
	X: 4 	Y: 10 	Time: 0.06985616683959961
	X: 4 	Y: 11 	Time: 0.06314635276794434
	X: 4 	Y: 12 	Time: 0.06342244148254395
	X: 4 	Y: 13 	Time: 0.06354689598083496
	X: 4 	Y: 14 	Time: 0.06470084190368652
	X: 4 	Y: 15 	Time: 0.06749296188354492
	X: 4 	Y: 16 	Time: 0.06827163696289062
	X: 4 	Y: 17 	Time: 0.06271767616271973
	X: 4 	Y: 18 	Time: 0.06446361541748047
	X: 4 	Y: 19 	Time: 0.06851792335510254
	X: 4 	Y: 20 	Time: 0.06246376037597656
	X: 4 	Y: 21 	Time: 0.06461238861083984
	X: 4 	Y: 22 	Time: 0.06641006469726562
	X: 4 	Y: 23 	Time: 0.06656289100646973
	X: 4 	Y: 24 	Time: 0.0626077651977539
	X: 4 	Y: 25 	Time: 0.06392264366149902
	X: 4 	Y: 26 	Time: 0.07279753684997559
	X: 4 	Y: 27 	Time: 0.06228041648864746
	X: 4 	Y: 28 	Time: 0.06278324127197266
	X: 4 	

	X: 4 	Y: 207 	Time: 0.06823158264160156
	X: 4 	Y: 208 	Time: 0.06237220764160156
	X: 4 	Y: 209 	Time: 0.06552481651306152
	X: 4 	Y: 210 	Time: 0.06872677803039551
	X: 4 	Y: 211 	Time: 0.0628502368927002
	X: 4 	Y: 212 	Time: 0.06273174285888672
	X: 4 	Y: 213 	Time: 0.06609320640563965
	X: 4 	Y: 214 	Time: 0.06933927536010742
	X: 4 	Y: 215 	Time: 0.06258487701416016
	X: 4 	Y: 216 	Time: 0.06399679183959961
	X: 4 	Y: 217 	Time: 0.07660293579101562
	X: 4 	Y: 218 	Time: 0.06389212608337402
	X: 4 	Y: 219 	Time: 0.06380653381347656
	X: 4 	Y: 220 	Time: 0.06273651123046875
	X: 4 	Y: 221 	Time: 0.0652306079864502
	X: 4 	Y: 222 	Time: 0.06755709648132324
	X: 4 	Y: 223 	Time: 0.06292533874511719
	X: 4 	Y: 224 	Time: 0.06299018859863281
	X: 4 	Y: 225 	Time: 0.08031129837036133
	X: 4 	Y: 226 	Time: 0.063720703125
	X: 4 	Y: 227 	Time: 0.06240582466125488
	X: 4 	Y: 228 	Time: 0.06555438041687012
	X: 4 	Y: 229 	Time: 0.0654606819152832
	X: 4 	Y: 230 	Time: 0.0634315013885498
	X: 4 	Y: 231 	Time: 0.06

	X: 5 	Y: 144 	Time: 0.06367754936218262
	X: 5 	Y: 145 	Time: 0.06280183792114258
	X: 5 	Y: 146 	Time: 0.06840658187866211
	X: 5 	Y: 147 	Time: 0.06425070762634277
	X: 5 	Y: 148 	Time: 0.06475186347961426
	X: 5 	Y: 149 	Time: 0.06769227981567383
	X: 5 	Y: 150 	Time: 0.06505155563354492
	X: 5 	Y: 151 	Time: 0.06482434272766113
	X: 5 	Y: 152 	Time: 0.06651568412780762
	X: 5 	Y: 153 	Time: 0.06468439102172852
	X: 5 	Y: 154 	Time: 0.06234169006347656
	X: 5 	Y: 155 	Time: 0.06387925148010254
	X: 5 	Y: 156 	Time: 0.06577515602111816
	X: 5 	Y: 157 	Time: 0.06959915161132812
	X: 5 	Y: 158 	Time: 0.06362462043762207
	X: 5 	Y: 159 	Time: 0.06743860244750977
	X: 5 	Y: 160 	Time: 0.06792092323303223
	X: 5 	Y: 161 	Time: 0.06343913078308105
	X: 5 	Y: 162 	Time: 0.06216096878051758
	X: 5 	Y: 163 	Time: 0.062226057052612305
	X: 5 	Y: 164 	Time: 0.06249117851257324
	X: 5 	Y: 165 	Time: 0.06362557411193848
	X: 5 	Y: 166 	Time: 0.06285262107849121
	X: 5 	Y: 167 	Time: 0.06902289390563965
	X: 5 	Y: 168 	

	X: 6 	Y: 80 	Time: 0.062418460845947266
	X: 6 	Y: 81 	Time: 0.06645941734313965
	X: 6 	Y: 82 	Time: 0.06540775299072266
	X: 6 	Y: 83 	Time: 0.06982183456420898
	X: 6 	Y: 84 	Time: 0.06322622299194336
	X: 6 	Y: 85 	Time: 0.06299018859863281
	X: 6 	Y: 86 	Time: 0.06374168395996094
	X: 6 	Y: 87 	Time: 0.06268835067749023
	X: 6 	Y: 88 	Time: 0.06326794624328613
	X: 6 	Y: 89 	Time: 0.06361889839172363
	X: 6 	Y: 90 	Time: 0.06382179260253906
	X: 6 	Y: 91 	Time: 0.06299018859863281
	X: 6 	Y: 92 	Time: 0.06667518615722656
	X: 6 	Y: 93 	Time: 0.06245708465576172
	X: 6 	Y: 94 	Time: 0.06401419639587402
	X: 6 	Y: 95 	Time: 0.0629429817199707
	X: 6 	Y: 96 	Time: 0.06271243095397949
	X: 6 	Y: 97 	Time: 0.07155036926269531
	X: 6 	Y: 98 	Time: 0.06340885162353516
	X: 6 	Y: 99 	Time: 0.06379485130310059
	X: 6 	Y: 100 	Time: 0.06832551956176758
	X: 6 	Y: 101 	Time: 0.06969451904296875
	X: 6 	Y: 102 	Time: 0.06769895553588867
	X: 6 	Y: 103 	Time: 0.06614971160888672
	X: 6 	Y: 104 	Time: 0.0769050121307

KeyboardInterrupt: 

In [ ]:
np.sum(l[l>0])
l

In [ ]:
import spams
import numpy as np

A = np.zeros(shape=(20, 10), dtype=np.single, order='F')
B = np.zeros(shape=(20, 1), dtype=np.single, order='F')
# B = np.zeros(shape=(20, 1), dtype=np.single, order='F')


alpha = spams.lasso(B, A, 
               return_reg_path = False, 
               lambda1 = 0.1, 
               lambda2 = 0.,
               pos = True,
               mode = 2,
               numThreads = -1,
              )


In [ ]:
type(alpha.A[:,0])

In [ ]:
alpha.toarray()[:,0]

In [ ]:

sitk.WriteImage(sitk.GetImageFromArray(fixedImage), "fixedImage.mhd")
sitk.WriteImage(sitk.GetImageFromArray(images[0]), "movingImage.mhd")
sitk.WriteImage(sitk.GetImageFromArray(labels[0]), "movinglabel.mhd")


In [ ]:
import SimpleITK as sitk
import numpy as np

fixedImage = sitk.GetArrayFromImage(sitk.ReadImage("fixedImage.mhd"))
image = sitk.GetArrayFromImage(sitk.ReadImage("movingImage.mhd"))
label = sitk.GetArrayFromImage(sitk.ReadImage("movinglabel.mhd"))

fixedImage = np.array(fixedImage, order='C')
images = np.array([image], order='C')
labels = np.array([label], order='C')

In [ ]:
print(fixedImage.shape)
print(images.shape)
print(labels.shape)

In [ ]:
from cGen import cGen

P = [3, 3, 3]
N = [5, 5, 5]
P = np.array(P, dtype=np.int32)
N = np.array(N, dtype=np.int32)

segmentationSPBM, segmentationSRC = cGen.applySPBMandSRCSpams(fixedImage, images, 
# segmentationSPBM, segmentationSRC = cGen.applySPBMandSRC(fixedImage, images, 
                              labels, 3, P, N, lassoTol=0.001, 
                              lassoMaxIter=1e4, verboseX=True,  verboseY=False, 
                              xmin=-1, xmax=-1, ymin=-1, ymax=-1, zmin=60, zmax=61)
                              # xmin=60, xmax=61, ymin=-1, ymax=-1, zmin=-1, zmax=-1)
                              # xmin=10, xmax=11, ymin=10, ymax=11, zmin=10, zmax=11)

In [ ]:
showImg_(sitk.GetImageFromArray(segmentationSPBM), 60)
showImg_(sitk.GetImageFromArray(segmentationSRC), 60)
# showImg_(sitk.GetImageFromArray(originalLabel), 60)
showImg_(sitk.GetImageFromArray(labels[0]), 60)

In [ ]:
segmentationSRC[segmentationSRC == 3]

In [ ]:

originalLabel = sitk.GetArrayFromImage(sitk.ReadImage(os.path.join(labelPath, testData[0])))
translatedSegmentationSPBM = translateToOriginal(segmentationSPBM, 
                                                 os.path.join(path, testData[0]), 
                                                 copyShape, 
                                                 offset, 
                                                 length)

print(Dice(originalLabel, translatedSegmentationSPBM, 3))

translatedSegmentationSRC = translateToOriginal(segmentationSRC, 
                                                os.path.join(path, testData[0]), 
                                                copyShape, 
                                                offset, 
                                                length)

print(Dice(originalLabel, translatedSegmentationSRC, 3))